In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader= torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer','dog','frog','horse','ship','truck')

import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(3 * 32 * 32, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 10)
        self.dropout = nn.Dropout(p=0.3)

    def forward(self,x):
        x = x.view(-1, 3 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = F.softmax(self.fc4(x))
        return x

net = Net()
torch.cuda.set_device(1)
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)
net.to(device)
summary(net, (3, 32, 32))

max_epoch = 50

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
optimizer = optim.RMSprop(net.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

for epoch in range(max_epoch):

    scheduler.step()
    running_loss = 0.0
    epoch_correct = 0
    epoch_total = 0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)

        _, predicted = torch.max(outputs.data,1)
        epoch_total += labels.size(0)
        epoch_correct += (predicted == labels).sum().item()

        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        # if i % 2000 == 1999:
        #     print ('[%d, %5d] loss: %3f' % (epoch+1, i+1, running_loss / 2000))
        #     running_loss = 0.0

    print('[epoch %d/%d] loss:%3f accuracy:%3f' % (epoch + 1, max_epoch, running_loss / i, 100 * epoch_correct / epoch_total))
print('Finished Training')

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy:%d %%' % (100 * correct / total))


Files already downloaded and verified
Files already downloaded and verified
cuda:1
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 256]         786,688
           Dropout-2                  [-1, 256]               0
            Linear-3                  [-1, 256]          65,792
           Dropout-4                  [-1, 256]               0
            Linear-5                  [-1, 256]          65,792
           Dropout-6                  [-1, 256]               0
            Linear-7                   [-1, 10]           2,570
Total params: 920,842
Trainable params: 920,842
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.01
Params size (MB): 3.51
Estimated Total Size (MB): 3.54
----------------------------------------------------------------


/home/jonric/.virtualenvs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[epoch 1/50] loss:2.359162 accuracy:10.422000
[epoch 2/50] loss:2.360300 accuracy:10.382000
[epoch 3/50] loss:2.350941 accuracy:11.300000
[epoch 4/50] loss:2.356281 accuracy:10.756000
[epoch 5/50] loss:2.362913 accuracy:10.112000
[epoch 6/50] loss:2.352512 accuracy:11.156000
[epoch 7/50] loss:2.360919 accuracy:10.316000
[epoch 8/50] loss:2.357343 accuracy:10.630000
[epoch 9/50] loss:2.362530 accuracy:10.138000
[epoch 10/50] loss:2.359213 accuracy:10.418000
